In [ ]:
import sys
import logging
import os
import math
import json
from tqdm import tqdm
import warnings
from IPython.display import display, Audio

import numpy as np
import librosa

import torch

import scdata

sys.path.append('..')
logging.getLogger().setLevel(logging.INFO)
warnings.filterwarnings('ignore')

from aural_travels.data import soundcloud
from aural_travels.train import visualizer

MODEL_DIR = '/home/leo/src/aural-travels/models/bottleneck_gen_pull0.25_max_dist1_enc27_dec1_layer_scale_dropout0.2'
DATA_DIR = '/home/leo/src/scdata'

In [ ]:
with open(os.path.join(MODEL_DIR, 'params.json')) as f:
    params = json.load(f)
    
params

In [ ]:
#split = 'training'
split = 'validation'

dataset = visualizer.load_dataset(params,
                                  split,
                                  torch.load(f'../models/encoding/{params["image_repr"]}/{split}.pt'))
image_repr = visualizer.create_image_repr(params)
model = visualizer.create_model(params, image_repr, dataset)
model.eval().to('cuda')

checkpoint = torch.load(os.path.join(MODEL_DIR, 'last_checkpoint.pt'))
model.load_state_dict(checkpoint['model'])

checkpoint['global_step']

In [ ]:
def show(image_seq, save=None):
    image = image_repr.tensor_to_image(image_repr.decode(image_seq)[0])
    display(image)
    if save:
        image.save(save)
    return image

corrupt_image_seq = image_repr.rand_image_seq(1, device='cuda')
show(corrupt_image_seq)

corrupt_image_seq = image_repr.rand_image_seq(1, device='cuda', patch_size=4)
show(corrupt_image_seq)

corrupt_image_seq = image_repr.zeros_image_seq(1, device='cuda')
show(corrupt_image_seq)

print('')

In [ ]:
top_k = 1
temperature = 1.0

indices = list(range(500, 600))

for idx in indices:
    track = dataset.tracks[idx]
    print(f'{track["genre"]}: {track["title"]} ({track["id"]})')
    
    audio1, ref_image_seq, audio2 = dataset[idx]
    
    audio1 = audio1[None, ...].to('cuda')
    audio2 = audio2[None, ...].to('cuda')

    ref_image_seq = ref_image_seq[None, ...].to('cuda')

    print(idx, 'ref')
    show(ref_image_seq)
    
    for i in range(1):
        image_seq = model.generate_image_seq(model.calc_audio_emb(audio1))
        show(image_seq)
        
        image_seq = model.generate_image_seq(model.calc_audio_emb(audio2))
        show(image_seq)

In [ ]:
from IPython.display import Audio

track_idx = 597
path = scdata.get_audio_path(os.path.join(DATA_DIR, 'audio'), dataset.tracks[track_idx]['id'])
print(path)
Audio(path)

In [ ]:
mel = torch.tensor(dataset.load_features(track_idx), dtype=torch.float)
mel = (mel - dataset.mfcc_mean) * dataset.mfcc_std_inv
print(mel.shape)
print(mel.shape[0] / 21)


from scipy.ndimage import gaussian_filter1d
from librosa.onset import onset_strength
from librosa.beat import beat_track

path = scdata.get_audio_path(os.path.join(DATA_DIR, 'audio'), dataset.tracks[track_idx]['id'])
y, _ = librosa.load(path, sr=dataset.sample_rate, mono=True)
onset_env = onset_strength(y, sr=dataset.sample_rate)
onset_env_filtered3 = gaussian_filter1d(onset_env, 3)
onset_env_filtered2 = gaussian_filter1d(onset_env, 2)
bpm, beats = beat_track(y, sr=dataset.sample_rate, units='time')

In [ ]:
#import msaf
#boundaries, labels = msaf.process(path)
#boundaries

In [ ]:
from IPython.display import clear_output
import time

from aural_travels import generate

frames_dir = os.path.join(MODEL_DIR, 'gen', 'frames') 
os.makedirs(frames_dir, exist_ok=True)

keyframes = generate.keyframes(model, mel, fps=23.0, device='cuda')
frames = generate.interpolate(model, keyframes, interframes=1)

start_time = time.time()

for i, frame in enumerate(frames):
    image = image_repr.tensor_to_image(frame)
    if i % 1 == 0:
        clear_output(wait=True)
        print(i, i/23, (i/23)/(time.time()-start_time))
        display(image)
        image.save(os.path.join(frames_dir, f'{i}.png'))

In [ ]:
model